<a href="https://colab.research.google.com/github/leidysguerrero120/Trabajo-Anal-tica-3---Marketing/blob/main/Preprocesamiento_Marketing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Analítica III - Caso de Estudio Marketing**
Leidys Guerrero Lobo. Leidys.guerrero@udea.edu.co
Paola Andrea Arabia Álvarez. Paola.arabia@udea.edu.co
Mateo Caicedo Aguirre. mateo.caicedo@udea.edu.co

***PREPROCESAMIENTO***

In [16]:
###Instalar la extensión ipython-sql
!pip install -q ipython-sql

###Cargar la extensión
%load_ext sql

###Instalar bibliotecas necesarias
!pip install sqlalchemy
!pip install mysql-connector-python

###Importar bibliotecas
import sqlite3
from sqlalchemy import create_engine
import pandas as pd


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [20]:
###Conectarse a la base de datos
conn = sqlite3.connect('/content/drive/MyDrive/db_movies_original')
###Crear un cursor
cur = conn.cursor()

In [21]:
###Información de las columnas en la tabla 'ratings'
cur.execute("PRAGMA table_info(ratings)")
columns_ratings = cur.fetchall()

###Nombre de la columna y su tipo de datos para la tabla 'ratings'
print("Tipos de datos de las columnas en la tabla 'ratings':")
for column in columns_ratings:
    print(column[1] + ": " + column[2])

###Información de las columnas de la tabla 'movies'
cur.execute("PRAGMA table_info(movies)")
columns_movies = cur.fetchall()

###Nombre de la columna y su tipo de datos para la tabla 'movies'
print("\nTipos de datos de las columnas en la tabla 'movies':")
for column in columns_movies:
    print(column[1] + ": " + column[2])

Tipos de datos de las columnas en la tabla 'ratings':
userId: INTEGER
movieId: INTEGER
rating: REAL
timestamp: INTEGER

Tipos de datos de las columnas en la tabla 'movies':
movieId: INTEGER
title: TEXT
genres: TEXT


***AJUSTES TABLA MOVIES***

In [24]:
###Extraer el año de lanzamiento de la película en la tabla movies, el cual está incluido en el título de película
###Agregar una nueva columna para almacenar el año

conn = sqlite3.connect('/content/drive/MyDrive/db_movies_original')
cursor = conn.cursor()

###Verificamos si la columna 'year' ya existe en la tabla 'movies'
cursor.execute("PRAGMA table_info(movies)")
columns = cursor.fetchall()
column_names = [col[1] for col in columns]
if 'year' not in column_names:
    ###Si no existe, agregamos la nueva columna
    cursor.execute("ALTER TABLE movies ADD COLUMN year INTEGER")

###Actualizar la nueva columna con el año extraído del título de la película
cursor.execute("""
    UPDATE movies
    SET year = CAST(SUBSTR(title, -5, 4) AS INTEGER)
""")

##Confirmamos los cambios y cerramos la conexión
conn.commit()
conn.close()

In [26]:
conn = sqlite3.connect('/content/drive/MyDrive/db_movies_original')
cursor = conn.cursor()

###Verificamos las columnas de la tabla 'movies'

###Obtenemos la información sobre las columnas de la tabla 'movies'
cursor.execute("PRAGMA table_info(movies)")
columns_info = cursor.fetchall()

###Imprimimos el nombre de la columna y su tipo de datos
for column in columns_info:
    print(column[1], "-", column[2])

###Cerramos la conexión a la base de datos
conn.close()

movieId - INTEGER
title - TEXT
genres - TEXT
year - INTEGER


In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


***AJUSTES EN LA TABLA RATINGS***

In [29]:
###Verificamos el contenido de las primeras filas de la tabla ratings
conn = sqlite3.connect('/content/drive/MyDrive/db_movies_original')

query = "SELECT * FROM ratings LIMIT 5"

###Ejecutamos la consulta y cargamos los resultados en un DataFrame de Pandas
ratings = pd.read_sql(query, conn)

print(ratings)
conn.close()

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


***CREACIÓN DE NUEVAS TABLAS***

In [31]:
conn = sqlite3.connect('/content/drive/MyDrive/db_movies_original')

###Crear tabla de usuarios con más de 50 películas vistas
query = """
    DROP TABLE IF EXISTS usuarios_sel;
    CREATE TABLE usuarios_sel AS
    SELECT "userId" AS user_id, COUNT(*) AS cnt_rat
    FROM ratings
    GROUP BY "userId"
    HAVING cnt_rat > 50
    ORDER BY cnt_rat DESC;
"""

conn.executescript(query)

conn = sqlite3.connect('/content/drive/MyDrive/db_movies_original')
query = "SELECT * FROM usuarios_sel LIMIT 5"

usuarios_sel = pd.read_sql(query, conn)

print(usuarios_sel)

conn.close()

   user_id  cnt_rat
0      414     2698
1      599     2478
2      474     2108
3      448     1864
4      274     1346


In [38]:
###Crear tabla con películas que han sido calificadas por más de 50 usuarios
conn = sqlite3.connect('/content/drive/MyDrive/db_movies_original')

query = """
DROP TABLE IF EXISTS movies_sel;

CREATE TABLE movies_sel AS
SELECT "movieId",
       COUNT(*) AS cnt_rat
FROM ratings
GROUP BY "movieId"
HAVING cnt_rat > 50
ORDER BY cnt_rat DESC;
"""

conn.executescript(query)
conn.close()

conn = sqlite3.connect('/content/drive/MyDrive/db_movies_original')
query = "SELECT * FROM movies_sel LIMIT 5"

movies_sel = pd.read_sql(query, conn)

print(movies_sel)

conn.close()

   movieId  cnt_rat
0      356      329
1      318      317
2      296      307
3      593      279
4     2571      278


In [41]:
###Crear tablas filtradas de calificaciones con usuarios y películas (ratings_final)
conn = sqlite3.connect('/content/drive/MyDrive/db_movies_original')

query = """
DROP TABLE IF EXISTS ratings_final;

CREATE TABLE ratings_final AS
SELECT a."userId" AS user_id,
       a."movieId" AS movie_id,
       a."rating" AS rating
FROM ratings a
INNER JOIN movies_sel b ON a.movieId = b.movieId
INNER JOIN usuarios_sel c ON a."userId" = c.user_id;
"""
conn.executescript(query)
conn.close()

conn = sqlite3.connect('/content/drive/MyDrive/db_movies_original')
query = "SELECT * FROM ratings_final LIMIT 5"
ratings_final = pd.read_sql(query, conn)

print(ratings_final)
conn.close()

   user_id  movie_id  rating
0        1         1     4.0
1        1         3     4.0
2        1         6     4.0
3        1        47     5.0
4        1        50     5.0


In [43]:
###Movies Final
conn = sqlite3.connect('/content/drive/MyDrive/db_movies_original')

query = """
DROP TABLE IF EXISTS movies_final;

CREATE TABLE movies_final AS
SELECT a.movieId AS movie_id,
       a.title AS movie_title,
       a.genres,
       a.year
FROM movies a
INNER JOIN movies_sel c ON a.movieId = c.movieId;
"""

conn.executescript(query)

conn.close()

conn = sqlite3.connect('/content/drive/MyDrive/db_movies_original')
query = "SELECT * FROM movies_final LIMIT 5"

movies_final = pd.read_sql(query, conn)

print(movies_final)
conn.close()

   movie_id              movie_title  \
0         1         Toy Story (1995)   
1         2           Jumanji (1995)   
2         3  Grumpier Old Men (1995)   
3         6              Heat (1995)   
4         7           Sabrina (1995)   

                                        genres  year  
0  Adventure|Animation|Children|Comedy|Fantasy  1995  
1                   Adventure|Children|Fantasy  1995  
2                               Comedy|Romance  1995  
3                        Action|Crime|Thriller  1995  
4                               Comedy|Romance  1995  


In [44]:
conn = sqlite3.connect('/content/drive/MyDrive/db_movies_original')
cursor = conn.cursor()

script = """
DROP TABLE IF EXISTS full_ratings;

CREATE TABLE full_ratings AS
SELECT a.*,
       c.movie_title,
       c.genres,
       c.year
FROM ratings_final a
INNER JOIN movies_final c ON a.movie_id = c.movie_id;
"""

cursor.executescript(script)

conn.commit()
conn.close()

print("Tabla 'full_ratings' creada con éxito.")

Tabla 'full_ratings' creada con éxito.


In [45]:
conn = sqlite3.connect('/content/drive/MyDrive/db_movies_original')

query = "SELECT * FROM full_ratings LIMIT 5"

full_ratings = pd.read_sql(query, conn)

print(full_ratings)

   user_id  movie_id  rating                  movie_title  \
0        1         1     4.0             Toy Story (1995)   
1        1         3     4.0      Grumpier Old Men (1995)   
2        1         6     4.0                  Heat (1995)   
3        1        47     5.0  Seven (a.k.a. Se7en) (1995)   
4        1        50     5.0   Usual Suspects, The (1995)   

                                        genres  year  
0  Adventure|Animation|Children|Comedy|Fantasy  1995  
1                               Comedy|Romance  1995  
2                        Action|Crime|Thriller  1995  
3                             Mystery|Thriller  1995  
4                       Crime|Mystery|Thriller  1995  


***Exportar la tabla preprocesada en SQL***

In [46]:
query = "SELECT * FROM full_ratings"

df = pd.read_sql_query(query, conn)

df.to_csv('full_ratings.csv', index=False)

print("La tabla 'full_ratings' se ha exportado como full_ratings.csv")

conn.close()

La tabla 'full_ratings' se ha exportado como full_ratings.csv


In [48]:
conn = sqlite3.connect('/content/drive/MyDrive/db_movies_original')

cur = conn.cursor()

cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cur.fetchall()

for table in tables:
    print(table[0])

ratings
movies
usuarios_sel
movies_sel
ratings_final
movies_final
full_ratings
